<span style="color:darkorange; font-size:60px">Learn Jupyter with Bloomberg</span></br>
<span style="color:lightblue; font-size:30px">Interactive data visualization - bqplot</span>

## A common analyst workflow

In [ ]:
import bqplot as bqp
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import bqplot.pyplot as blt
from utils.plot_utils import set_style # Helper function for nicer looking plots.

In [ ]:
# Styling for dark background matplotlib charts
set_style()

### Creating some data
#### Monte Carlo Simulation
A simple method for simulation the evolution of an asset price.

$$\Large S_{t} = S_0 e^{(r - 0.5\sigma^2)dt + \sigma \sqrt{dt}z}$$
$$ z \sim N(0, 1)$$

In [ ]:
# Some parameters
S_0 = 100 # Asset price at inceptiom
r = 0.01 # "Risk free" rate (interest rate)
sigma = 0.05 # Volatility (standard deviation of price)
paths = 1000 # Number of simulations
steps = 252 # Number of steps
years = 5 # Time horizon for simulation
dt = years/252 # Time step

In [ ]:
# Simulation
S = np.zeros((steps, paths)) # Matrix of paths and their steps. Columns are paths, rows steps.
S[0] = S_0 # Initial price
for i in range(1, steps):
    # Applying the formula above in a row-wise fshion
    S[i] = S[i-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.standard_normal(paths))

<span style="color: orange">So what does it look like?</span>

In [ ]:
# Uncomment to see the output
# pd.DataFrame(S)

In [ ]:
# pd.DataFrame(S).plot(legend=False, figsize=(16,9));

### Why do we need to visualize our data?
There are many reasons, of course, but a few important ones which come to mind are:
1. <span style="color: orange; font-size:16px">Easier to read.</span> Think of the stock price of a company. If you want to see how the company has been doing over time, it's harder to try and read through the data in a tabular format. A line chart makes achieving this task much easier.
2. <span style="color: orange; font-size:16px">Telling a story.</span>  "A picture is worth a thousand words" may sound like a cliche, but it really is not. A chart well done can convey a much more compelling story than narration.
3. <span style="color: orange; font-size:16px">Summary statistics and other information which is derived from data can be very misleading.</span>  Point in case "Datasaurus".

[Check out the Datasaurus example](datasaurus.ipynb)

### Data visualization is, of course, not limited to charts!

Check out this [great resource](https://informationisbeautiful.net/data/) for novel ways of visualizing data. (link last retrieved 2020-10-31)

### Common chart types used in data analysis
1. Bar
2. Line
3. Scatter
4. Pie
5. Heatmap
6. Choropleth (Geo)

This is by no means an exhaustive list, there are too many to list here, but these are the most common ones! We will focus on the first three and how to use them in `bqplot`

### Static vs. Dynamic Charts
We always want to use the right tool for the job. What are the differences?

- Static chart images are great for reports and other contexts where the image doesn't change frequently. Typically generated in some backend are saved to a file with come compression technique. 
- Dynamic charts do not produce an image (hand wavey), but rather draw objects in some context (like a web browser) which together compose a chart. (They can be saved to an image too, if needed). **Most importantly, they allow for _*interactions*_!** 

In [ ]:
# Some fake data
np.random.seed(0)
x_data = np.arange(1,21,1)
y_fake_line = 100 + np.random.normal(0,1,20).cumsum()
y_fake_dots = np.random.normal(1,10,20)

### Static charts of some fake data

In [ ]:
# Using matplotlib
fig, axes = plt.subplots(1,2, figsize=(16,4))
axes[0].scatter(x_data, y_fake_dots)
axes[1].plot(x_data, y_fake_line);

### Dynamic charts of some fake data

In [ ]:
# Using bqplot
blt.plot(x_data, y_fake_line)
blt.show()

<span style="color: orange; font-size:20px">Dynamic, interactive charts are excellent for applications, live dashboards and reproducible research!</span>

### The "Grammar of Graphics" in 30 seconds, a _*very*_ light introduction.
Look at the chart above. It is made of a number of components. Can you identify those components?

- There is data to begin with, illustrated by the line mark.
- Axes, both x and y.
- Scales for those axes, both linear (other contexts can have ordinal, log etc.).
- The figure object itself, which combines all these elements.
- There is some coordinate system here.

We are trying to tell a story with our data by taking all of the point above (and more) into consideration. The Grammar of Graphics is quite a dense book to read but there are many resources online with summaries.

`bqplot` is based on the Grammar of Graphics!

## Let's dive in!

In [ ]:
def generate_fake_stock_price(steps):
    return (np.arange(steps), 100 + np.random.normal(0,1,steps).cumsum())

In [ ]:
# Data
x_data, y_data = generate_fake_stock_price(100)

# Creating the bqplot chart objects
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()

line = bqp.Lines(x=x_data, y=y_data, labels=['Fake stock price'], display_legend=True,
                 scales={'x': sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='Index')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='y-value')

fig = bqp.Figure(marks=[line], axes=[ax_x, ax_y], title='Line Chart')
fig

### Interactivity
<span style="color: orange; font-size:18px">Everything is an object in `bqplot`</span>.

This is crucial for interactivity. Everything is drawn directly in the browser.

In [ ]:
# Checking out the line mark object
line.x

In [ ]:
line.y

#### Let's create a separate window for the bqplot chart (great JupyterLab feature)

Now we can continue exploring the different elements of the chart!

In [ ]:
# Not only retrieving data!
line.x, line.y = generate_fake_stock_price(100)

### A 'delight' factor - do you like animations?

In [ ]:
fig.animation_duration=1000 # transition animation in milliseconds

<span style="color: orange; font-size:18px">We can change anything we want on the fly, and it will update immediately!</span>

Let's try changing a few more properties.

In [ ]:
fig.title = 'A new title for my chart'

In [ ]:
line.colors = ['limegreen']

<span style="color: orange; font-size:18px">Built for the widgets ecosystem</span>

Let's build a small app to change the line data and color randomly.

In [ ]:
# Callback function for the button
def change_line(click=None):
    line.x, line.y = generate_fake_stock_price(100)
    line.colors = [bqp.CATEGORY10[np.random.randint(len(bqp.CATEGORY10))]]
    fig.title = "Line chart with color {}".format(line.colors)
    
    
btn = widgets.Button(description='Change line!', button_style='warning')
btn.on_click(change_line)
widgets.VBox([btn, fig])

In [ ]:
# If we don't like the legend
line.display_legend=False

#### Scatter Charts

In [ ]:
# Scales for x and y axes
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()

# Tooltip widget - new addition
ttip = bqp.Tooltip(fields=['x', 'y'], formats=['', '.2f'])

# Scatter mark
scatt = bqp.Scatter(x=x_data[:20], y=y_fake_dots, scales={'x': sc_x, 'y': sc_y},  labels=['Random scatter data'],
                    colors=bqp.CATEGORY10, display_legend=True, tooltip=ttip)

# Axes for both x and y
ax_x = bqp.Axis(scale=sc_x, label='Date')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', tick_format='0.0f')

# Figure object
fig = bqp.Figure(marks=[scatt], axes=[ax_x, ax_y], title='Scatter Chart', animation_duration=1000)
fig

#### Colorful and nice, but we are looking for a trend.
Simple OLS regression.

In [ ]:
regression = np.polyfit(x_data[:20], y_fake_dots, deg=2)
polyfit = np.polyval(regression, x_data)

# Creating a line mark
ols_line = bqp.Lines(x=x_data[:20], y=polyfit, scales={'x':sc_x, 'y':sc_y}, colors=['pink'])

# Adding the regression line
fig.marks = [scatt, ols_line]

#### Okay, we have a trend line we can see. What if we could play with the data and regression parameters and see what effect it would have on the trajectory?

<span style="color: orange; font-size:18px">We can interact with our data directly with `bqplot` by clicking and draggin it. It's as easy as that.</span>

In [ ]:
scatt.enable_move = True

<span style="color: orange; font-size:18px">Try it! Now let's make a small app out of it.</span>

In [ ]:
# Some widgets
slider_scatt = widgets.IntSlider(value=2, min=1, max=5)

# Callback for linear regression degree of freedom change
def scatter_callback(click=None):
    # New regression
    regression = np.polyfit(scatt.x, scatt.y, deg=slider_scatt.value)
    polyfit = np.polyval(regression, scatt.x)
    
    with ols_line.hold_sync():
        ols_line.x = scatt.x
        ols_line.y = polyfit

# Listening for change events
scatt.observe(scatter_callback, names=['x'])
scatt.observe(scatter_callback, names=['y'])
slider_scatt.observe(scatter_callback, 'value')

widgets.VBox([slider_scatt, fig])